In [11]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
# Initialize the SentenceTransformer model
model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)

<All keys matched successfully>


In [5]:
# Initialize the Qdrant client
client = QdrantClient(url="http://localhost:6333")
collection = "knowledge_repo_768"

In [10]:
query = 'search_query: how can I deprecate probably?'
query_embeddings = model.encode([query])
search_result = client.query_points(
    collection_name=collection,
    query=query_embeddings[0],
    with_payload=True,
    limit=10
).points

contexts = [point.payload["doc"].removeprefix("search_document: ") for point in search_result]
context_string = " ".join([doc for doc in contexts])

prompt = f"""Use the following pieces of context to answer the question at the end.
    {context_string}
    Question: {query}
"""


Use the following pieces of context to answer the question at the end.
    - Deprecation tools can be used to prevent back-sliding to old usage - Def:
    - Don't have a deadline
    - Aren't high priority for the org
  - Goal: advertising the existence of a new system
  - When: new system offers compelling benefits
- Compulsory deprecation:
  - Def: usually comes with a deadline for the removal of obsolete system
  - How to scale: done by 1 team of expert. Advs:
    - Reuse expertise
    - Reduce burden to users - Why: summary of what you expect someone to take away from the doc
- Structure: should have beginning, middle & end:
  - Beginning: problem
  - Middle: recommended solution
  - End: takeaways
- Deprecation:
  - Should avoid abandoned docs as much as possible
  - Deprecation steps:
    - Remove or mark it as obsolete
    - Indicate where to go for new info (if possible) - Static analysis
    - Alerting/deprecation warnings: should be:
      - Actionable: specify how to migrate

In [12]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "developer", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="To deprecate effectively, you should follow a structured approach based on the context provided. Here’s how to do it:\n\n### Beginning: Identify the Problem\nFirst, understand the necessity of deprecation. This could be due to redundancy, complexity, deprecated functionality, or the introduction of a new system that offers compelling benefits. You want to ensure that users are migrating away from obsolete systems in a way that doesn't negatively impact operations. \n\n### Middle: Recommended Solutions\n1. **Determine the Type of Deprecation**: Identify whether you're opting for advisory deprecation (no immediate removal) or compulsory deprecation (with a deadline for removal).\n\n2. **Establish Clear Communication**:\n   - Remove or mark obsolete functionality to avoid confusion.\n   - Provide clear, actionable migration paths, explaining to users how they can transition to the new system or functionality.\n   - Ensure alerts and deprecation warnings are 